<a href="https://colab.research.google.com/github/LaurensSluyterman/Bootstrapped_Deep_Ensembles/blob/master/PBPK_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab contains all the necessary code in order to calcualte a confidence interval for a geometric mean ratio as is advocated in our paper  "*Towards more RobustEvaluation of the Predictive Performance of Physiologically Based Pharmacokinetic Models: using Confidence Intervals to support use of Model-informed Dosing in Clinical Care*", which is currently under review.

In [15]:
#@title The code that is used ot create the intervals
import numpy as np
import scipy
from scipy import stats

def confidence_interval_individual_values(observations, predictions,
                                          alpha=0.1):
    """This function calculates a confidence interval using the individual
    predictions.

    For details on the reasoning behind the interval, please see our
    paper: ....

    Arguments:
      observations: An array containing the individual observations
      predictions: An array containing the individual predictions
      alpha: A float that determines the significance of the confidence interval.
        The significance is (1-alpha/2)*100%.

    Return:
      None
    """
    # Some preliminary checks
    if len(observations) != len(predictions):
      print('different number of predictions and observations')
      return
    if alpha > 1:
      print('alpha cannot be larger than 1. For a 90% confidence interval, use alpha=0.1')
    # Convert to logscale
    error_values = np.log(predictions) - np.log(observations)

    # Calcualte a CI on logscale for the mean of the error
    N = len(error_values)
    var = np.var(error_values)
    average = np.mean(error_values)
    t = scipy.stats.t(N-1).ppf(1-alpha/2)
    loglowerbound = average - t * np.sqrt(var / N)
    logupperbound = average + t * np.sqrt(var / N)

    # Convert the CI back to the original scale
    CI = [np.exp(loglowerbound), np.exp(logupperbound)]

    # Print the result
    print(f'{100*(1-alpha)}% confidence interval: {CI}')

def confidence_interval(GM_observed, GM_predicted, GCV, N, alpha=0.1):
    """This function calculates a confidence interval using the observed
       geometric mean, the predicted geometric mean, and the coefficient
       of variation.

    Arguments:
      GM_observed (float): The geometric mean of the observations
      GM_predicted (float): The geometric mean of the predictions
      GCV (float): The geometric coefficient of variation. Note that values are
        often reported as percentiles in literature. If 70% is reported, use 0.7.
      N (int): The number of observations.
      alpha (float): Determines the significance of the confidence interval.
        The significance is (1-alpha/2)*100%.

    Return:
      None
    """
    average = np.log(GM_observed)
    var = np.log(GCV**2 + 1)
    t = scipy.stats.t(N-1).ppf(1-alpha/2)
    loglowerbound = average + t * np.sqrt(var / N)
    logupperbound = average - t * np.sqrt(var / N)
    CI = [GM_predicted / np.exp(loglowerbound), GM_predicted / np.exp(logupperbound)]
    print(f'{100*(1-alpha)}% confidence interval: {CI}')


The implementation of the code is give in the code block above. The some code can be found in the module CIs.py in the Confidence_intervals_code folder. The examples that follow load the code from that module. There is no need to run the code manually first.

## Example when individual observations and predictions are available

The cell below illustrates how a confidence interval can be created using individual observations and predicitons. The hypothetical values can be replaced by relevant values. By default, a 90% confidence interval is given. To get, for instance, a 95% confidence interval, change alpha to 0.05.


In [16]:
from CIs import confidence_interval_individual_values

hypothetical_observations = [132,
                             111,
                             120,
                             190,
                             115,
                             130,
                             ]

hypothetical_predictions = [110,
                            121,
                            125,
                            170,
                            125,
                            130,
                            ]

confidence_interval_individual_values(hypothetical_observations,
                                      hypothetical_predictions,
                                      alpha=0.1)


90.0% confidence interval: [0.9080389449267313, 1.0711936588331088]


# Example second approach

Quite often, only the geometric mean and coefficient of variation are published and not the individual AUC values. In this case, the first approach is not possible and the second approach must be used.

In [ ]:
from CIs import confidence_interval
hypothetical_observed_GM = 120
hypothetical_predicted_GM = 120
hypothetical_CV = 0.4 # N.B. 40% should be entered as 0.4!
Number_of_subjects = 20
confidence_interval(hypothetical_observed_GM, hypothetical_predicted_GM,
                    hypothetical_CV, N=Number_of_subjects, alpha=0.1)

90.0% confidence interval: [0.8616065596990528, 1.1606225471975127]
